# Greeks Calculator & Hedger - Testing Notebook

This notebook allows you to test all components of the application without loading the UI.

**Components covered:**
- DataLoader: Fetch market data, options chains, rates, volatility surface
- GreeksCalculator: Compute Black-Scholes greeks for positions
- PortfolioAggregator: Aggregate portfolio-level metrics
- ScenarioAnalyzer: Calculate P&L under different market scenarios
- HedgeOptimizer: Optimize hedge portfolio to minimize risk

## Setup

In [1]:
import sys
import os

# Add src directory to path
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
project_root = os.path.dirname(notebook_dir)
src_path = os.path.join(project_root, 'src')
if src_path not in sys.path:
    sys.path.insert(0, src_path)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Data directory
DATA_DIR = os.path.join(project_root, 'data')
os.makedirs(DATA_DIR, exist_ok=True)

print(f"Project root: {project_root}")
print(f"Data directory: {DATA_DIR}")

Project root: /Users/mdabdullahalmahin/Desktop/Projects/greeks-calculator-and-hedger
Data directory: /Users/mdabdullahalmahin/Desktop/Projects/greeks-calculator-and-hedger/data


In [2]:
# Import all components
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from data_loader import DataLoader
from greeks_calculator import GreeksCalculator
from portfolio_aggregator import PortfolioAggregator
from scenario_analyzer import ScenarioAnalyzer
from hedge_optimizer import HedgeOptimizer

print("All components imported successfully!")

All components imported successfully!


---
## 1. DataLoader - Fetch Market Data

The DataLoader handles fetching and caching:
- Stock prices and dividends from Yahoo Finance
- Risk-free rates from FRED/yfinance
- Options chains with implied volatility
- Volatility surfaces

In [3]:
# Initialize DataLoader
data_loader = DataLoader(data_dir=DATA_DIR, cache_expiry_hours=1)

# Define symbols to work with
SYMBOLS = ['AAPL', 'MSFT', 'GOOGL', 'NVDA', 'SPY']
print(f"Working with symbols: {SYMBOLS}")

Working with symbols: ['AAPL', 'MSFT', 'GOOGL', 'NVDA', 'SPY']


In [4]:
# Fetch stock data (prices, dividends, borrow costs)
stock_data = data_loader.fetch_stock_data(SYMBOLS, use_cache=True)
print("\n--- Stock Data ---")
display(stock_data[['symbol', 'spot_price', 'dividend_yield', 'borrow_cost_bps', 'transaction_cost_bps']])

Fetching stock data for 5 symbols
  AAPL: transaction_cost=64.37 bps (base=2.0, spread=50, vol=0, cap=0, volatility=12.37); borrow_cost=252.0 bps (base=10.0, spread=200, vol=0, cap=0, short=0, div=42.0, htb=0.0)
  MSFT: transaction_cost=22.11 bps (base=2.0, spread=7.82, vol=0, cap=0, volatility=12.29); borrow_cost=105.64 bps (base=10.0, spread=15.64, vol=0, cap=0, short=0, div=80.0, htb=0.0)
  GOOGL: transaction_cost=21.02 bps (base=2.0, spread=6.68, vol=0, cap=0, volatility=12.34); borrow_cost=49.37 bps (base=10.0, spread=13.37, vol=0, cap=0, short=0, div=26.0, htb=0.0)
  NVDA: transaction_cost=69.26 bps (base=2.0, spread=50, vol=0, cap=0, volatility=17.26); borrow_cost=410.0 bps (base=10.0, spread=200, vol=0, cap=0, short=0, div=200.0, htb=0.0)
  SPY: transaction_cost=16.5 bps (base=2.0, spread=2.5, vol=0, cap=0, volatility=12.0); borrow_cost=127.0 bps (base=10.0, spread=10.0, vol=0, cap=0, short=0, div=107.0, htb=0.0)
Fetched stock data for 5 symbols

--- Stock Data ---


,symbol,spot_price,dividend_yield,borrow_cost_bps,transaction_cost_bps
0,AAPL,246.70,0.0042,252.00,64.37
1,MSFT,454.52,0.0080,105.64,22.11
2,GOOGL,322.00,0.0026,49.37,21.02
3,NVDA,178.07,0.0200,410.00,69.26
4,SPY,677.58,0.0107,127.00,16.50


In [5]:
# Fetch risk-free rates
rates = data_loader.fetch_risk_free_rates(use_cache=True)
print("\n--- Risk-Free Rates ---")
display(rates)

Fetching risk-free rates
Using yfinance fallback for Treasury rates
Fetched 9 risk-free rate tenors

--- Risk-Free Rates ---


,tenor_days,rate
0,30,0.03572
1,90,0.03572
2,180,0.03572
3,365,0.03572
4,730,0.03858
5,1095,0.03858
6,1825,0.03858
7,2555,0.03858
8,3650,0.04295


In [6]:
# Fetch options chain for a single symbol
options_chain = data_loader.fetch_options_chain('AAPL', use_cache=True)
print(f"\n--- Options Chain for AAPL ({len(options_chain)} contracts) ---")
if not options_chain.empty:
    display(options_chain.head(10))

Fetching options chain for AAPL
Fetched 2226 options for AAPL

--- Options Chain for AAPL (2226 contracts) ---


,symbol,expiry,strike,option_type,lastPrice,volume,openInterest,impliedVolatility,bid,ask
0,AAPL,2026-01-23,145.0,call,106.07,69.0,0.0,0.00001,0.0,0.0
2,AAPL,2026-01-23,155.0,call,92.66,2.0,0.0,0.00001,0.0,0.0
3,AAPL,2026-01-23,170.0,call,80.63,1.0,0.0,0.00001,0.0,0.0
5,AAPL,2026-01-23,180.0,call,81.25,1.0,0.0,0.00001,0.0,0.0
6,AAPL,2026-01-23,185.0,call,71.26,2.0,0.0,0.00001,0.0,0.0
7,AAPL,2026-01-23,190.0,call,59.81,16.0,0.0,0.00001,0.0,0.0
8,AAPL,2026-01-23,195.0,call,62.31,1.0,0.0,0.00001,0.0,0.0
9,AAPL,2026-01-23,200.0,call,47.05,14.0,0.0,0.00001,0.0,0.0
10,AAPL,2026-01-23,205.0,call,44.21,80.0,0.0,0.00001,0.0,0.0
11,AAPL,2026-01-23,210.0,call,41.87,17.0,0.0,0.00001,0.0,0.0


In [7]:
# Build volatility surface
vol_surface = data_loader.build_volatility_surface(SYMBOLS[:3], use_cache=True)
print(f"\n--- Volatility Surface ({len(vol_surface)} points) ---")
if not vol_surface.empty:
    display(vol_surface.head(10))

Building volatility surface for 3 symbols
Fetching stock data for 3 symbols
  AAPL: transaction_cost=64.37 bps (base=2.0, spread=50, vol=0, cap=0, volatility=12.37); borrow_cost=252.0 bps (base=10.0, spread=200, vol=0, cap=0, short=0, div=42.0, htb=0.0)
  MSFT: transaction_cost=22.11 bps (base=2.0, spread=7.82, vol=0, cap=0, volatility=12.29); borrow_cost=105.64 bps (base=10.0, spread=15.64, vol=0, cap=0, short=0, div=80.0, htb=0.0)
  GOOGL: transaction_cost=21.02 bps (base=2.0, spread=6.68, vol=0, cap=0, volatility=12.34); borrow_cost=49.37 bps (base=10.0, spread=13.37, vol=0, cap=0, short=0, div=26.0, htb=0.0)
Fetched stock data for 3 symbols
Using cached options chain for AAPL
Fetching options chain for MSFT
Fetched 2605 options for MSFT
Fetching options chain for GOOGL
Fetched 2927 options for GOOGL
Built volatility surface with 6663 points

--- Volatility Surface (6663 points) ---


,symbol,expiry,strike,moneyness,implied_vol
54,AAPL,2026-01-23,130.0,0.526956,0.500005
55,AAPL,2026-01-23,135.0,0.547223,0.500005
56,AAPL,2026-01-23,140.0,0.567491,0.500005
0,AAPL,2026-01-23,145.0,0.587758,0.000010
57,AAPL,2026-01-23,150.0,0.608026,0.500005
1,AAPL,2026-01-23,155.0,0.628293,0.000010
2,AAPL,2026-01-23,170.0,0.689096,0.000010
58,AAPL,2026-01-23,175.0,0.709364,0.500005
3,AAPL,2026-01-23,180.0,0.729631,0.000010
59,AAPL,2026-01-23,180.0,0.729631,0.500005


In [8]:
# Generate synthetic positions for testing
positions = data_loader.generate_synthetic_positions(SYMBOLS, num_positions=20, seed=42)
print(f"\n--- Synthetic Positions ({len(positions)} positions) ---")
display(positions)

Using cached stock data for 5 symbols

--- Synthetic Positions (20 positions) ---


,position_id,symbol,quantity,instrument_type,strike,expiry,option_type
0,POS_0001,NVDA,-87,option,184.97,2026-06-21,call
1,POS_0002,GOOGL,-2,option,348.80,2026-07-19,put
2,POS_0003,NVDA,89,option,155.52,2026-12-30,put
3,POS_0004,AAPL,62,option,226.19,2026-11-17,put
4,POS_0005,GOOGL,-140,equity,NaN,None,None
5,POS_0006,SPY,14,option,545.66,2026-05-19,call
6,POS_0007,MSFT,-35,option,488.02,2026-11-10,put
7,POS_0008,AAPL,399,equity,NaN,None,None
8,POS_0009,MSFT,62,option,504.54,2026-07-31,put
9,POS_0010,NVDA,471,equity,NaN,None,None


In [9]:
# Load all data at once (recommended for full pipeline)
all_data = data_loader.load_all_data(
    symbols=SYMBOLS,
    num_positions=20,
    seed=42,
    use_cache=True,
    generate_positions=True,
)
print("\n--- All Data Loaded ---")
for key, df in all_data.items():
    if df is not None:
        print(f"{key}: {len(df)} rows")

Loading all required data
Using cached stock data for 5 symbols
Saved market_data.csv (5 rows)
Using cached risk-free rates
Saved rates.csv (9 rows)
Building volatility surface for 5 symbols
Using cached stock data for 5 symbols
Using cached options chain for AAPL
Using cached options chain for MSFT
Using cached options chain for GOOGL
Fetching options chain for NVDA
Fetched 3474 options for NVDA
Fetching options chain for SPY
Fetched 7466 options for SPY
Built volatility surface with 15600 points
Saved vol_surface.csv (15600 rows)
Generating 20 synthetic positions
Using cached stock data for 5 symbols
Saved positions.csv (20 rows)
Generated 20 positions
Data loading complete

--- All Data Loaded ---
market_data: 5 rows
rates: 9 rows
vol_surface: 15600 rows
positions: 20 rows


---
## 2. GreeksCalculator - Compute Position Greeks

Computes Black-Scholes greeks for options and equity positions:
- Delta, Gamma, Vega, Theta, Rho
- Position-level greeks (unit greeks × quantity)

In [10]:
# Initialize GreeksCalculator
greeks_calc = GreeksCalculator(data_dir=DATA_DIR)

In [11]:
# Test single option greeks calculation
single_greeks = greeks_calc.compute_black_scholes_greeks(
    spot=150.0,
    strike=155.0,
    time_to_expiry=0.25,  # 3 months
    rate=0.05,
    volatility=0.25,
    option_type='call',
    dividend_yield=0.01
)
print("\n--- Single Option Greeks (AAPL-like call, 3mo, strike 155) ---")
for greek, value in single_greeks.items():
    print(f"  {greek}: {value:.6f}")


--- Single Option Greeks (AAPL-like call, 3mo, strike 155) ---
  delta: 0.451184
  gamma: 0.021072
  vega: 29.632488
  theta: -17.226205
  rho: 15.433683


In [12]:
# Test put option
put_greeks = greeks_calc.compute_black_scholes_greeks(
    spot=150.0,
    strike=145.0,
    time_to_expiry=0.25,
    rate=0.05,
    volatility=0.25,
    option_type='put',
    dividend_yield=0.01
)
print("\n--- Single Option Greeks (AAPL-like put, 3mo, strike 145) ---")
for greek, value in put_greeks.items():
    print(f"  {greek}: {value:.6f}")


--- Single Option Greeks (AAPL-like put, 3mo, strike 145) ---
  delta: -0.338695
  gamma: 0.019483
  vega: 27.398016
  theta: -11.440444
  rho: -13.833028


In [13]:
# Run the full greeks pipeline
positions_with_greeks = greeks_calc.run_pipeline(validate=True)
print(f"\n--- Positions with Greeks ({len(positions_with_greeks)} positions) ---")
display(positions_with_greeks[['position_id', 'symbol', 'instrument_type', 'quantity', 
                               'delta', 'gamma', 'vega', 'theta', 'rho',
                               'position_delta', 'position_gamma']].head(10))


Greeks Validation Warnings:
  Info: Found 1 long options with positive theta (>0.01). This can occur for deep ITM puts with high dividend yields (mathematically valid).

--- Positions with Greeks (20 positions) ---


,position_id,symbol,instrument_type,quantity,delta,gamma,vega,theta,rho,position_delta,position_gamma
0,POS_0001,NVDA,option,-87,0.358817,0.026046,42.429042,-7.338212,24.884211,-31.217103,-2.266012
1,POS_0002,GOOGL,option,-2,-0.564195,0.005323,88.405402,-22.252133,-109.118670,1.128390,-0.010646
2,POS_0003,NVDA,option,89,-0.094802,0.007793,28.944915,-1.629381,-16.764502,-8.437352,0.693597
3,POS_0004,AAPL,option,62,-0.145913,0.008191,51.049208,-2.681639,-31.296417,-9.046621,0.507836
4,POS_0005,GOOGL,equity,-140,1.000000,0.000000,0.000000,0.000000,0.000000,-140.000000,-0.000000
5,POS_0006,SPY,option,14,0.861368,0.001407,83.290158,-61.586677,139.798996,12.059152,0.019700
6,POS_0007,MSFT,option,-35,-0.644550,0.007252,149.822697,-2.373619,-261.495786,22.559264,-0.253810
7,POS_0008,AAPL,equity,399,1.000000,0.000000,0.000000,0.000000,0.000000,399.000000,0.000000
8,POS_0009,MSFT,option,62,-0.826130,0.006157,82.770848,2.119778,-219.503976,-51.220080,0.381735
9,POS_0010,NVDA,equity,471,1.000000,0.000000,0.000000,0.000000,0.000000,471.000000,0.000000


In [14]:
# Validate greeks
validation_results = greeks_calc.validate_greeks(positions_with_greeks, verbose=True)
print("\n--- Validation Results ---")
for check, passed in validation_results.items():
    status = "✓" if passed else "✗"
    print(f"  {status} {check}")


Greeks Validation Warnings:
  Info: Found 1 long options with positive theta (>0.01). This can occur for deep ITM puts with high dividend yields (mathematically valid).

--- Validation Results ---
  ✓ gamma_non_negative
  ✓ atm_call_delta
  ✓ vega_non_negative
  ✗ long_theta_negative
  ✓ equity_delta_one


---
## 3. PortfolioAggregator - Portfolio-Level Metrics

Aggregates position-level greeks to portfolio-level:
- Total portfolio greeks
- Breakdown by symbol
- Breakdown by instrument type
- Top risky positions

In [15]:
# Initialize PortfolioAggregator
portfolio_agg = PortfolioAggregator(data_dir=DATA_DIR)

In [16]:
# Aggregate portfolio greeks
portfolio_summary = portfolio_agg.aggregate_portfolio_greeks(positions_with_greeks)
print("\n--- Portfolio Summary ---")
for key, value in portfolio_summary.items():
    if isinstance(value, float):
        print(f"  {key}: {value:,.2f}")
    else:
        print(f"  {key}: {value}")


--- Portfolio Summary ---
  total_delta: 3.66
  total_gamma: -2.12
  total_vega: -9,419.41
  total_theta: 3,716.65
  total_rho: -43,882.96
  total_notional: 1,428,632.49
  num_positions: 20


In [17]:
# Breakdown by symbol
symbol_breakdown = portfolio_agg.aggregate_by_symbol(positions_with_greeks)
print("\n--- Breakdown by Symbol ---")
display(symbol_breakdown)


--- Breakdown by Symbol ---


,symbol,delta,gamma,vega,theta,rho,notional,num_positions
0,MSFT,-1674.660816,0.127925,-112.001817,214.502904,-4456.894020,792228.36,4
1,NVDA,1110.658514,-2.201641,-3541.082146,785.960642,-15008.249906,265858.51,5
2,AAPL,846.899064,0.933720,3644.730528,-118.651970,-5760.525907,252867.50,5
3,GOOGL,-291.296279,-0.999868,-10577.121993,3697.050110,-20614.477798,108192.00,5
4,SPY,12.059152,0.019700,1166.062205,-862.213476,1957.185949,9486.12,1


In [18]:
# Breakdown by instrument type
instrument_breakdown = portfolio_agg.aggregate_by_instrument_type(positions_with_greeks)
print("\n--- Breakdown by Instrument Type ---")
display(instrument_breakdown)


--- Breakdown by Instrument Type ---


,instrument_type,delta,gamma,vega,theta,rho,notional,num_positions
0,equity,299.000000,0.000000,0.000000,0.000000,0.000000,1244285.05,8
1,option,-295.340365,-2.120165,-9419.413223,3716.648209,-43882.961682,184347.44,12


In [19]:
# Identify top risks
top_risks = portfolio_agg.identify_top_risks(positions_with_greeks, top_n=5)
print("\n--- Top 5 Risky Positions (by absolute delta) ---")
display(top_risks)


--- Top 5 Risky Positions (by absolute delta) ---


,position_id,symbol,instrument_type,option_type,quantity,position_delta,position_gamma,position_vega,position_theta,position_rho
0,POS_0019,MSFT,equity,NaN,-873,-873.0,-0.0,-0.0,-0.0,-0.0
1,POS_0012,MSFT,equity,NaN,-773,-773.0,-0.0,-0.0,-0.0,-0.0
2,POS_0011,NVDA,equity,NaN,758,758.0,0.0,0.0,0.0,0.0
3,POS_0020,AAPL,equity,NaN,494,494.0,0.0,0.0,0.0,0.0
4,POS_0010,NVDA,equity,NaN,471,471.0,0.0,0.0,0.0,0.0


In [20]:
# Full summary report
full_report = portfolio_agg.generate_summary_report(positions_with_greeks)
print("\n--- Full Report Keys ---")
for key in full_report.keys():
    print(f"  - {key}")


--- Full Report Keys ---
  - portfolio_summary
  - symbol_breakdown
  - instrument_type_breakdown
  - top_risks


---
## 4. ScenarioAnalyzer - P&L Scenarios

Calculate portfolio P&L under different market scenarios using greeks approximation:
- Price changes
- Volatility changes
- Interest rate changes
- Time decay

In [21]:
# Initialize ScenarioAnalyzer
scenario_analyzer = ScenarioAnalyzer(data_dir=DATA_DIR)

In [22]:
# Test scenario: Market drops 5%, vol increases 10%, rates up 25bps, 7 days pass
scenario_result = scenario_analyzer.calculate_scenario_pnl(
    price_change_pct=-5.0,
    vol_change_pct=10.0,
    rate_change_bps=25,
    time_decay_days=7,
    portfolio_summary=portfolio_summary,
    positions=positions_with_greeks
)

print("\n--- Scenario: Market Down 5%, Vol Up 10%, Rates +25bps, 7 Days ---")
print(f"  Total P&L: ${scenario_result['total_pnl']:,.2f}")
print(f"    Delta P&L: ${scenario_result['delta_pnl']:,.2f}")
print(f"    Gamma P&L: ${scenario_result['gamma_pnl']:,.2f}")
print(f"    Vega P&L: ${scenario_result['vega_pnl']:,.2f}")
print(f"    Theta P&L: ${scenario_result['theta_pnl']:,.2f}")
print(f"    Rho P&L: ${scenario_result['rho_pnl']:,.2f}")


--- Scenario: Market Down 5%, Vol Up 10%, Rates +25bps, 7 Days ---
  Total P&L: $-1,294.48
    Delta P&L: $-56.99
    Gamma P&L: $-257.11
    Vega P&L: $-941.94
    Theta P&L: $71.28
    Rho P&L: $-109.71


In [23]:
# Display scenario breakdown table
print("\n--- Scenario Breakdown ---")
display(scenario_result['breakdown'])


--- Scenario Breakdown ---


,Component,P&L ($),Input Change,Greek Exposure
0,Delta,-56.994328,-5.0%,3.66
1,Gamma,-257.114999,-5.0%,-2.1202
2,Vega,-941.941322,+10.0%,"-9,419.41"
3,Theta,71.278185,7 days,"3,716.65"
4,Rho,-109.707404,+25 bps,"-43,882.96"
5,Total,-1294.479869,Combined,-


In [24]:
# Test multiple scenarios
scenarios = [
    {'name': 'Bull Case', 'price': 10.0, 'vol': -5.0, 'rate': 0, 'days': 7},
    {'name': 'Bear Case', 'price': -10.0, 'vol': 15.0, 'rate': 0, 'days': 7},
    {'name': 'Flat + Time Decay', 'price': 0.0, 'vol': 0.0, 'rate': 0, 'days': 30},
    {'name': 'Vol Spike', 'price': 0.0, 'vol': 25.0, 'rate': 0, 'days': 0},
    {'name': 'Rate Hike', 'price': 0.0, 'vol': 0.0, 'rate': 50, 'days': 0},
]

print("\n--- Multi-Scenario Analysis ---")
scenario_results = []
for s in scenarios:
    result = scenario_analyzer.calculate_scenario_pnl(
        price_change_pct=s['price'],
        vol_change_pct=s['vol'],
        rate_change_bps=s['rate'],
        time_decay_days=s['days'],
        portfolio_summary=portfolio_summary,
        positions=positions_with_greeks
    )
    scenario_results.append({
        'Scenario': s['name'],
        'Price Δ': f"{s['price']:+.1f}%",
        'Vol Δ': f"{s['vol']:+.1f}%",
        'Rate Δ': f"{s['rate']:+d}bps",
        'Days': s['days'],
        'Total P&L': f"${result['total_pnl']:,.2f}"
    })

display(pd.DataFrame(scenario_results))


--- Multi-Scenario Analysis ---


,Scenario,Price Δ,Vol Δ,Rate Δ,Days,Total P&L
0,Bull Case,+10.0%,-5.0%,+0bps,7,$-372.22
1,Bear Case,-10.0%,+15.0%,+0bps,7,"$-2,484.08"
2,Flat + Time Decay,+0.0%,+0.0%,+0bps,30,$305.48
3,Vol Spike,+0.0%,+25.0%,+0bps,0,"$-2,354.85"
4,Rate Hike,+0.0%,+0.0%,+50bps,0,$-219.41


---
## 5. HedgeOptimizer - Optimize Hedges

Find optimal hedges to neutralize portfolio risk:
- Build hedge universe (stocks, ETFs, bonds, options)
- Optimize for delta-neutral, rho-neutral targets
- Minimize transaction and borrow costs

In [25]:
# Initialize HedgeOptimizer
hedge_optimizer = HedgeOptimizer(data_dir=DATA_DIR)

In [26]:
# Build hedge universe
hedge_universe = hedge_optimizer.build_hedge_universe(
    symbols=SYMBOLS,
    config={
        'include_etfs': True,
        'etf_symbols': ['SPY', 'QQQ', 'IWM'],
        'include_ir_instruments': True,
        'treasury_symbols': ['TLT', 'IEF', 'SHY'],
        'use_market_data': True
    }
)
print(f"\n--- Hedge Universe ({len(hedge_universe)} instruments) ---")
display(hedge_universe[['symbol', 'instrument_type', 'spot_price', 'delta_per_unit', 'rho_per_unit']])


--- Hedge Universe (10 instruments) ---


,symbol,instrument_type,spot_price,delta_per_unit,rho_per_unit
0,AAPL,equity,246.70,1.0,0.000000
1,MSFT,equity,454.52,1.0,0.000000
2,GOOGL,equity,322.00,1.0,0.000000
3,NVDA,equity,178.07,1.0,0.000000
4,SPY,etf,677.58,1.0,0.000000
5,QQQ,etf,100.00,1.0,0.000000
6,IWM,etf,100.00,1.0,0.000000
7,TLT,bond,86.65,0.0,-14.520492
8,IEF,bond,95.55,0.0,-6.905898
9,SHY,bond,82.80,0.0,-1.595222


In [27]:
# Load portfolio exposures
portfolio_exposures = hedge_optimizer.load_portfolio_exposures()
print("\n--- Current Portfolio Exposures ---")
print(f"  Total Delta: {portfolio_exposures['total_delta']:,.2f}")
print(f"  Total Rho: {portfolio_exposures['total_rho']:,.2f}")
print(f"  Total Notional: ${portfolio_exposures['total_notional']:,.2f}")
print(f"  Num Positions: {portfolio_exposures['num_positions']}")


--- Current Portfolio Exposures ---
  Total Delta: 3.66
  Total Rho: -43,882.96
  Total Notional: $1,428,632.49
  Num Positions: 20


In [28]:
# Load market data
try:
    market_data = hedge_optimizer.load_market_data()
except FileNotFoundError:
    market_data = pd.DataFrame()

In [29]:
# Define hedge targets
targets = {
    'delta_target': 0.0,        # Target delta-neutral
    'delta_tolerance': 100.0,   # Allow +/- 100 delta
    'rho_target': 0.0,          # Target rho-neutral
    'rho_tolerance': 5000.0    # Allow +/- 5000 rho
}

print("\n--- Hedge Targets ---")
for key, value in targets.items():
    print(f"  {key}: {value:,.2f}")


--- Hedge Targets ---
  delta_target: 0.00
  delta_tolerance: 100.00
  rho_target: 0.00
  rho_tolerance: 5,000.00


In [30]:
# Run hedge optimization
hedge_recommendations, optimization_summary = hedge_optimizer.optimize_hedge_portfolio(
    portfolio_exposures=portfolio_exposures,
    hedge_universe=hedge_universe,
    market_data=market_data,
    targets=targets,
    holding_period_years=1.0
)

print("\n--- Optimization Summary ---")
print(f"  Solver Status: {optimization_summary['solver_status']}")
print(f"  Total Hedge Cost: ${optimization_summary['total_hedge_cost']:,.2f}")
print(f"  Residual Delta: {optimization_summary['residual_delta']:,.2f}")
print(f"  Residual Rho: {optimization_summary['residual_rho']:,.2f}")
print(f"  Hedge Effectiveness: {optimization_summary['hedge_effectiveness_pct']:.1f}%")
print(f"  Number of Trades: {optimization_summary['num_hedge_trades']}")


--- Optimization Summary ---
  Solver Status: optimal
  Total Hedge Cost: $11,668.67
  Residual Delta: 0.01
  Residual Rho: -5,000.00
  Hedge Effectiveness: 93.7%
  Number of Trades: 9


In [31]:
# Display hedge recommendations
if not hedge_recommendations.empty:
    print("\n--- Hedge Recommendations ---")
    display(hedge_recommendations[['symbol', 'instrument_type', 'side', 'hedge_quantity', 
                                   'delta_contribution', 'rho_contribution', 'estimated_cost']])
else:
    print("\nNo hedge trades recommended (portfolio may already be near target).")


--- Hedge Recommendations ---


,symbol,instrument_type,side,hedge_quantity,delta_contribution,rho_contribution,estimated_cost
0,AAPL,equity,buy,0.000268,0.000268,0.000000,0.000425
1,MSFT,equity,buy,0.000665,0.000665,0.000000,0.000669
2,GOOGL,equity,buy,0.003065,0.003065,0.000000,0.002075
3,SPY,etf,buy,0.038715,0.038715,0.000000,0.043284
4,QQQ,etf,sell,-1.849127,-1.849127,-0.000000,0.462282
5,IWM,etf,sell,-1.845468,-1.845468,-0.000000,0.461367
6,TLT,bond,sell,-2677.801047,-0.000000,38882.989309,11654.708238
7,IEF,bond,sell,-0.001252,-0.000000,0.008643,0.004860
8,SHY,bond,buy,0.022737,0.000000,-0.036271,0.001645


In [32]:
# Save hedge tickets (optional)
# hedge_optimizer.save_hedge_tickets(hedge_recommendations)
# hedge_optimizer.save_optimization_summary(optimization_summary)

---
## 6. Full End-to-End Pipeline

Run the complete pipeline from data loading to hedge optimization.

In [33]:
# Run the full end-to-end pipeline
hedge_recs, opt_summary = hedge_optimizer.run_end_to_end(
    symbols=SYMBOLS,
    targets=targets,
    hedge_config={
        'include_etfs': True,
        'etf_symbols': ['SPY', 'QQQ'],
        'include_ir_instruments': True,
        'use_market_data': True
    },
    save_results=False,  # Set to True to save results
    holding_period_years=1.0
)

Hedge Optimization Pipeline

Loading portfolio exposures...
  Delta: 3.66, Rho: -43,882.96, Positions: 20

Top 5 Risky Positions (by absolute delta):
  MSFT: -873.00 delta
  MSFT: -773.00 delta
  NVDA: 758.00 delta
  AAPL: 494.00 delta
  NVDA: 471.00 delta

Building hedge universe...
  Instruments: 9

Loading market data...
  Loaded: 5 symbols

Optimizing...
  Status: optimal, Trades: 8, Cost: $11,685.32
  Residual Delta: -0.02, Rho: -4,999.90, Effectiveness: 93.5%

Complete


---
## 7. Custom Testing Section

Use this section for your own custom tests.

In [34]:
# Custom test: Manual position creation and greeks calculation
custom_positions = pd.DataFrame([
    {
        'position_id': 'CUSTOM_001',
        'symbol': 'AAPL',
        'quantity': 100,
        'instrument_type': 'equity',
        'strike': None,
        'expiry': None,
        'option_type': None
    },
    {
        'position_id': 'CUSTOM_002',
        'symbol': 'AAPL',
        'quantity': -10,
        'instrument_type': 'option',
        'strike': 200.0,
        'expiry': (datetime.now() + timedelta(days=60)).strftime('%Y-%m-%d'),
        'option_type': 'call'
    },
    {
        'position_id': 'CUSTOM_003',
        'symbol': 'MSFT',
        'quantity': 5,
        'instrument_type': 'option',
        'strike': 400.0,
        'expiry': (datetime.now() + timedelta(days=90)).strftime('%Y-%m-%d'),
        'option_type': 'put'
    }
])

print("\n--- Custom Positions ---")
display(custom_positions)


--- Custom Positions ---


,position_id,symbol,quantity,instrument_type,strike,expiry,option_type
0,CUSTOM_001,AAPL,100,equity,NaN,None,None
1,CUSTOM_002,AAPL,-10,option,200.0,2026-03-22,call
2,CUSTOM_003,MSFT,5,option,400.0,2026-04-21,put


In [35]:
# Save custom positions and run greeks calculation
custom_positions.to_csv(os.path.join(DATA_DIR, 'positions.csv'), index=False)

# Run greeks calculation on custom positions
custom_greeks_calc = GreeksCalculator(data_dir=DATA_DIR)
custom_positions_with_greeks = custom_greeks_calc.run_pipeline(validate=True)

print("\n--- Custom Positions with Greeks ---")
display(custom_positions_with_greeks[['position_id', 'symbol', 'instrument_type', 'quantity',
                                       'delta', 'gamma', 'vega', 'theta',
                                       'position_delta', 'position_gamma', 'position_vega']])


Greeks Validation: All checks passed ✓

--- Custom Positions with Greeks ---


,position_id,symbol,instrument_type,quantity,delta,gamma,vega,theta,position_delta,position_gamma,position_vega
0,CUSTOM_001,AAPL,equity,100,1.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000
1,CUSTOM_002,AAPL,option,-10,0.999313,0.000003,0.003788,-6.068831,-9.993129,-0.000031,-0.037880
2,CUSTOM_003,MSFT,option,5,-0.013515,0.001233,7.767361,-1.815955,-0.067573,0.006167,38.836805


In [36]:
# Aggregate custom portfolio
custom_agg = PortfolioAggregator(data_dir=DATA_DIR)
custom_summary = custom_agg.aggregate_portfolio_greeks(custom_positions_with_greeks)

print("\n--- Custom Portfolio Summary ---")
for key, value in custom_summary.items():
    if isinstance(value, float):
        print(f"  {key}: {value:,.4f}")
    else:
        print(f"  {key}: {value}")


--- Custom Portfolio Summary ---
  total_delta: 89.9393
  total_gamma: 0.0061
  total_vega: 38.7989
  total_theta: 51.6085
  total_rho: -329.0772
  total_notional: 29,409.6000
  num_positions: 3


---
## 8. Quick Reference

### DataLoader Methods
- `fetch_stock_data(symbols)` - Get stock prices, dividends, borrow costs
- `fetch_risk_free_rates()` - Get Treasury rates
- `fetch_options_chain(symbol)` - Get options chain for a symbol
- `build_volatility_surface(symbols)` - Build volatility surface
- `generate_synthetic_positions(symbols, num_positions)` - Generate test positions
- `load_all_data(symbols)` - Load everything at once

### GreeksCalculator Methods
- `compute_black_scholes_greeks(spot, strike, tte, rate, vol, opt_type)` - Single option greeks
- `run_pipeline()` - Full greeks calculation pipeline
- `validate_greeks(positions)` - Validate computed greeks

### PortfolioAggregator Methods
- `aggregate_portfolio_greeks(positions)` - Total portfolio greeks
- `aggregate_by_symbol(positions)` - Breakdown by symbol
- `aggregate_by_instrument_type(positions)` - Breakdown by type
- `identify_top_risks(positions, top_n)` - Top risky positions

### ScenarioAnalyzer Methods
- `calculate_scenario_pnl(price_change, vol_change, rate_change, time_decay)` - Calculate P&L

### HedgeOptimizer Methods
- `build_hedge_universe(symbols, config)` - Build available hedge instruments
- `optimize_hedge_portfolio(exposures, universe, market_data, targets)` - Find optimal hedges
- `run_end_to_end(symbols, targets)` - Full pipeline